In [88]:
import pandas as pd
import requests
import json

In [223]:
def get_uniprot_from_pdb_entity(pdb_id, entity_id = 1):
    """
    Fetches the UniProt accession and name for a given PDB ID and entity ID using the PDBe API.
    Where multiple mappings exist, the mapping with the highest coverage is returned.
    """
    url = f"https://www.ebi.ac.uk/pdbe/api/v2/pdb/entry/uniprot_mapping/{pdb_id}/{entity_id}"
    response = requests.get(url)
    response.raise_for_status()
    if response.status_code == 200:
        data = response.json()
        if pdb_id in data:
            mapping = [{"accession": item['accession'], "name": item['name'], "coverage": item['residues'][0]['endIndex'] - item['residues'][0]['startIndex']} for item in data[pdb_id]['data']]
            max_mapping = max(mapping, key=lambda x: x['coverage'])
            if max_mapping:
                accession = max_mapping['accession']
                name = max_mapping['name']
                return accession, name
        raise ValueError(f"Unexpected response structure: {data}")
    # return None

def get_ec_from_uniprot(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}?fields=ec"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    if response.status_code == 200:
        data = response.json()
        protein_description = data.get('proteinDescription', {}).get('recommendedName', {})
        if 'ecNumbers' in protein_description:
            ec_values = [ec['value'] for ec in protein_description['ecNumbers']]
            return ec_values
        else:
            return None
    raise ValueError(f"Unexpected response structure: {data}")

def get_alphafold_structure(uniprot_id):
    #TODO: Return none on 404
    """
    Returns the URL of the AlphaFold structure for a given UniProt ID, or None if not available.
    """
    url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        if response.status_code == 200:
            data = response.json()
            for item in data:
                if item.get("uniprotAccession", None) == uniprot_id:
                    cif_url = item.get("cifUrl", None)
                    if cif_url:
                        return cif_url
        raise ValueError(f"Unexpected response structure: {data}")
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return None
        else:
            raise e    
    

In [ ]:
#First write a function to get the uniprot ID from kahraman dataset (assume that no _1 suffix means entity ID 1)

kahraman_table_1 = pd.read_csv("kahraman_dataset_table1_updated.tsv", sep = "\t")

kahraman_table_1[["uniprot_accession", "uniprot_name"]] = kahraman_table_1.apply(lambda row: get_uniprot_from_pdb_entity(row["updated_pdb_id"], row["entity_id"]), axis=1, result_type="expand")

kahraman_table_1["uniprot_ec"] = kahraman_table_1["uniprot_accession"].apply(lambda x: get_ec_from_uniprot(x) if pd.notna(x) else None)

kahraman_table_1["af_cif_url"] = kahraman_table_1["uniprot_accession"].apply(lambda x: get_alphafold_structure(x) if pd.notna(x) else None)

kahraman_table_1.loc[kahraman_table_1["af_cif_url"].notna(), "af_cif_filename"] = kahraman_table_1.loc[kahraman_table_1["af_cif_url"].notna(), "af_cif_url"].apply(lambda x: x.split('/')[-1])


#make the cif directory if it doesn't exist
import os
if not os.path.exists("kahraman_af_structures"):
    os.makedirs("kahraman_af_structures")

for idx, row in kahraman_table_1.loc[kahraman_table_1["af_cif_url"].notna()].iterrows():

    cif_url = row["af_cif_url"]
    cif_filename = row["af_cif_filename"]
    structure_directory = "kahraman_af_structures"
    cif_path = os.path.join(structure_directory, cif_filename)
    if not os.path.exists(cif_path):
        response = requests.get(cif_url, stream=True)
        response.raise_for_status()

        with open(cif_path, "wb") as cif_file:
            for chunk in response.iter_content(chunk_size=8192):
                cif_file.write(chunk)

#check that all files were downloaded
for idx, row in kahraman_table_1.loc[kahraman_table_1["af_cif_url"].notna()].iterrows():
    cif_filename = row["af_cif_filename"]
    cif_path = os.path.join("kahraman_af_structures", cif_filename)
    if not os.path.exists(cif_path):
        print(f"File {cif_path} not found!")


#now we have a directory of input structures, we need to make the structure manifest for the analysis run - accession, filename and structure directory

structure_manifest = kahraman_table_1.loc[kahraman_table_1["af_cif_filename"].notna(), ["uniprot_accession", "af_cif_filename"]]
structure_manifest["structure_directory"] = "kahraman_af_structures"

structure_manifest.to_csv("kahraman_af_structure_manifest.tsv", header=None index=False)




In [75]:
input_uniprots.loc[input_uniprots["AF"].isin(all_structures["accession"]) == False]

,0,AF
9,P00969,AF-P00969-F1-model_4
74,P00588,AF-P00588-F1-model_4


These two structures are a bacterial protein and diptheria protein, both with no alphafold DB structures, hence they are missing from the outputs of the analysis.

In [76]:
total_input_proteins = input_uniprots["AF"].nunique() - 2

In [77]:
print(f"Total number of input proteins: {total_input_proteins}")

Total number of input proteins: 101


In [78]:
all_structures = pd.read_csv("kahraman_alphacognate_cathaf/combined_structure_summaries.tsv.gz", sep="\t")

no_transplants = all_structures.loc[all_structures.num_transplants == 0]
print(f"Total number of structures with no transplants: {no_transplants.accession.nunique()}")

success = all_structures.loc[all_structures.num_transplants > 0]
print(f"Total number of structures with transplants (any): {success.accession.nunique()}")


Total number of structures with no transplants: 60
Total number of structures with transplants (any): 41


So first of all, we were only able to transplant any ligands to 40% of the dataset. Why is that? The ProCogGraph dataset of structures/ligands is limited to enzyme structures in the PDB, and many of the proteins in the Kahraman dataset are non-enzymes.

In [ ]:
all_transplants = pd.read_csv("kahraman_alphacognate_cathaf/combined_transplants.tsv.gz", sep="\t")

In [40]:
all_transplants.accession.nunique() #.loc[all_transplants.top_ranked == True]

40

In [20]:
all_transplants.loc[(all_transplants.accession == "AF-P00963-F1-model_4") & (all_transplants.top_ranked == True)]

,accession,transplant_structure,foldseek_rmsd,global_rmsd,local_rmsd,ligand,ligand_het_code,ligand_name,ligand_chain,ligand_residues,...,cognate_mapping_smiles,cognate_mapping_xref,cluster,cluster_center,Score,Type,nrgrank_runtime,top_ranked,transplanted_chain_id,nrgrank_tcs
4116,AF-P00963-F1-model_4,6chd_bio-h_A,6.868,6.867851,1.334752,6chd_bm1_C,KAA,5'-O-[(L-LYSYLAMINO)SULFONYL]ADENOSINE,T,601,...,Cc1ccc(C(=O)OP(=O)(O)OCC2OC(n3cnc4c(N)ncnc43)C...,Pubchem:102515309|KEGG:C21460|CHEBI:91232,2.0,"-0.8359554015638798,1.3352133429718456,-3.0370...",-1921856,ligand,191.055988,1,CH,0.871002
4136,AF-P00963-F1-model_4,6ilh_bio-h_A,6.764,6.764052,2.901763,6ilh_bm1_C,KAA,5'-O-[(L-LYSYLAMINO)SULFONYL]ADENOSINE,V,601,...,Cc1ccc(C(=O)OP(=O)(O)OCC2OC(n3cnc4c(N)ncnc43)C...,Pubchem:102515309|KEGG:C21460|CHEBI:91232,2.0,"-0.8359554015638798,1.3352133429718456,-3.0370...",-1921856,ligand,191.055988,1,CH,0.871002
